In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
from PIL import Image,ImageOps
import tensorflow as tf
import cv2
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
# from keras_frcnn import RoiPoolingConv
# from keras_rcnn.layers import RoiPoolingConv

In [18]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])

    image=preprocess_input(image)
    # image = tf.keras.utils.to_categorical(image.numpy())
    # image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image
# def divide(x,y):
#   return x / 255,y
# def normalize_image(image,y):
#     mean = [0.485, 0.456, 0.406]
#     std = [0.229, 0.224, 0.225]
#     image -= mean
#     image /= std
#     return image,y

In [7]:
dataset=pd.read_csv("/kaggle/input/labels/labels.csv")
dataset

,image_name,points
0,medicakolkata_mau_mon--2_2022_6_8_6_8_51.jpeg,"[459.5, 169.5, 886.0999755859375, 172.19999694..."
1,kamalnayanbajaj_micu_mon--22_2022_6_20_17_47_2...,"[422.4, 158, 967.8, 135.5, 957.3, 545.2, 409.5..."
2,kamalnayanbajaj_micu_mon--15_2022_6_20_22_46_2...,"[497.4, 184.9, 792.7999877929688, 229.19999694..."
3,medicakolkata_ccu2_mon--3_2022_5_30_12_6_19.jpeg,"[353.79998779296875, 112.69999694824219, 764.2..."
4,medicakolkata_ccu2_mon--5_2022_6_8_6_6_37.jpeg,"[246.9, 155.3, 756.5999755859375, 205.89999389..."
...,...,...
1995,medicakolkata_ccu3_mon--5_2022_6_8_5_2_54.jpeg,"[479.9, 207.1, 1006.5, 190.4, 998.6, 518.8, 47..."
1996,hcgnagpur_icu_mon--7_2022_6_20_0_30_58.jpeg,"[403.9, 245.8, 720.2999877929688, 261.60000610..."
1997,hcgnagpur_icu_mon--5_2022_5_18_21_30_27.jpeg,"[355.1000061035156, 181.3000030517578, 646.3, ..."
1998,medicakolkata_ccu1_mon--4_2022_6_8_19_0_39.jpeg,"[386.4, 148.5, 879.6, 165.8, 866.6, 550.4, 373..."


In [8]:
Dataset=dataset.to_numpy()
X_train=Dataset[:,0]
Y_train=Dataset[:,1]

In [9]:
for i in range (Y_train.shape[0]):
    Y_train[i]=Y_train[i].split(",")
for i in range (Y_train.shape[0]):
    Y_train[i]=np.array(Y_train[i])
for i in range (Y_train.shape[0]):
    Y_train[i][7]=Y_train[i][7].replace(']','')
    Y_train[i][0]=Y_train[i][0].replace('[','')
Y_train=np.array(Y_train)
for i in range (Y_train.shape[0]):
    Y_train[i] = Y_train[i].astype(float)
for i in range (Y_train.shape[0]):
    Y_train[i] = Y_train[i].astype(int) 

In [10]:
Y=np.zeros([2000,8])
for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
        Y[i,j]=Y_train[i][j]

In [27]:
batch_size = 32
img_height = 720
img_width = 1280
ds = tf.data.Dataset.from_tensor_slices('/kaggle/input/images/images/'+X_train)
ds = ds.map(load_and_preprocess_image)
labels = tf.data.Dataset.from_tensor_slices(Y)
dataset = tf.data.Dataset.zip((ds, labels))
# dataset=dataset.map(divide)
# dataset=dataset.map(normalize_image)
# dataset = dataset.batch(batch_size)
dataset = dataset.shuffle(buffer_size=32)
num_images=2000
num_train=int(num_images*0.8)
train_data=dataset.take(num_train)
test_data=dataset.skip(num_train).take(int(num_images*0.2))
dataset = dataset.batch(batch_size)
train_data = train_data.batch(batch_size)
test_data = test_data.batch(batch_size)

In [24]:
# from efficientnet.keras import EfficientNetB4
from keras.layers import Dense, GlobalAveragePooling2D,Conv2D,DepthwiseConv2D
from keras.models import Model

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
x = base_model.output
x = Conv2D(1280,(3,3), activation='selu')(x)
x = Conv2D(1540,(3,3), activation='relu')(x)
x = Conv2D(1960,(3,3), activation='selu')(x)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(8, activation='linear')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='adam', loss='mean_squared_error',metrics='RootMeanSquaredError')
# model.fit(train_data, epochs=50)

In [25]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_3 (Rescaling)         (None, 224, 224, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
normalization_3 (Normalization) (None, 224, 224, 3)  7           rescaling_3[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization_3[0][0]            
____________________________________________________________________________________________

In [28]:
model.fit(train_data,epochs=50,validation_data=test_data)

Epoch 1/50
50/50 [==============================] - 24s 479ms/step - loss: 56.7756 - root_mean_squared_error: 7.5350 - val_loss: 73.6471 - val_root_mean_squared_error: 8.5818
Epoch 2/50
50/50 [==============================] - 23s 461ms/step - loss: 75.8351 - root_mean_squared_error: 8.7083 - val_loss: 158.6926 - val_root_mean_squared_error: 12.5973
Epoch 3/50
50/50 [==============================] - 23s 462ms/step - loss: 157.1098 - root_mean_squared_error: 12.5343 - val_loss: 278.9455 - val_root_mean_squared_error: 16.7017
Epoch 4/50
50/50 [==============================] - 23s 460ms/step - loss: 92.2302 - root_mean_squared_error: 9.6037 - val_loss: 108.8573 - val_root_mean_squared_error: 10.4335
Epoch 5/50
50/50 [==============================] - 23s 456ms/step - loss: 142.4479 - root_mean_squared_error: 11.9352 - val_loss: 127.1199 - val_root_mean_squared_error: 11.2747
Epoch 6/50
50/50 [==============================] - 23s 453ms/step - loss: 132.6017 - root_mean_squared_error: 11